In [1]:
!pip install kafka-python
!pip install pyspark
!pip install confluent-kafka
!pip install streamlit
!pip install kafka

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=e1ac09e40bcd0745e44624cc13bf3d81fbba380906fa1099896416b0d6e1a787
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.4 

In [ ]:
from kafka import KafkaProducer
from confluent_kafka import Consumer, KafkaError, Producer
import streamlit as st
import matplotlib.pyplot as plt
from datetime import datetime
import time
import random

# Kafka producer configuration
bootstrap_servers = 'pkc-6ojv2.us-west4.gcp.confluent.cloud:9092'
sasl_username = 'PBLZTYNSYXUNEDWJ'
sasl_password = 'TFVC+XYaBB9b2WkCBF8OnzrzNpA+MHtg/r/PJhMq9aX6wJR7hGHJavGPVIx8QEqK'
topic_name = 'network-traffic'

producer = KafkaProducer(
    bootstrap_servers=bootstrap_servers,
    security_protocol='SASL_SSL',
    sasl_mechanism='PLAIN',
    sasl_plain_username=sasl_username,
    sasl_plain_password=sasl_password
)

# Generate and publish network traffic data to Kafka topic
while True:
    # Generate random network traffic data
    source_ip = '.'.join(str(random.randint(0, 255)) for _ in range(4))
    destination_ip = '.'.join(str(random.randint(0, 255)) for _ in range(4))
    bytes_sent = random.randint(1000, 100000)

    # Publish network traffic data to Kafka topic
    producer.send(topic_name, f"{source_ip},{destination_ip},{bytes_sent}".encode('utf-8'))

    # Wait for 1 second before generating next network traffic data
    time.sleep(1)

# Start SparkSession
spark = SparkSession.builder.appName("Real-Time Network Traffic Analysis for Telecommunications").getOrCreate()

# Define schema for the data
schema = StructType([StructField("source_ip", StringType(), True), StructField("destination_ip", StringType(), True), StructField("bytes_sent", IntegerType(), True), StructField("event_time", TimestampType(), True) ])

# Create Kafka consumer
configuration conf = {'bootstrap.servers': bootstrap_servers,
                      'security.protocol': 'SASL_SSL',
                      'sasl.mechanisms': 'PLAIN',
                      'sasl.username': sasl_username,
                      'sasl.password': sasl_password,
                      'group.id': 'network-traffic-group',
                      'auto.offset.reset': 'earliest' }


# Create Kafka consumer
consumer = Consumer(conf)

# Subscribe to the Kafka topic
consumer.subscribe([kafka_topic])

# Read data from Kafka and perform real-time visualization
def read_from_kafka():
  # Create a Streamlit app
  st.title("Real-Time Network Traffic Analysis for Telecommunications")

  # Create a plot to visualize processed data
  fig, ax = plt.subplots()

  # Initialize an empty list to store processed data and event times
  processed_data = []
  event_times = []


# Function to process incoming Kafka messages and update the plot
def process_message(message):
  nonlocal processed_data, event_times
  if message is None:
    return
    if message.error():
      if message.error().code() == KafkaError._PARTITION_EOF:
        return
      else:
        print(f"Error: {message.error()}")
        return
  value = message.value().decode('utf-8')
  data = value.split(',')
  source_ip = data[0]
  destination_ip = data[1]
  bytes_sent = int(data[2])
  event_time = datetime.now()
  row = (source_ip, destination_ip, bytes_sent, event_time)
  df = spark.createDataFrame([row], schema=schema)

# Perform window-based aggregations
aggregated_df = df.groupBy(window("event_time", "1 minute"), "source_ip").agg(sum("bytes_sent").alias("total_bytes_sent")).orderBy(desc("total_bytes_sent"))

# Convert the aggregated DataFrame to Pandas DataFrame for plotting
pd_df = aggregated_df.toPandas()

# Process the Kafka message and update the plot
processed_data.append(bytes_sent)
event_times.append(event_time)
ax.clear()
ax.plot(event_times, processed_data)
ax.set_xlabel("Event Time")
ax.set_ylabel("Information")
st.pyplot(fig)

# Continuously read messages from Kafka topic
while True:
  message = consumer.poll(1.0)
  if message is None:
    continue
  if message.error():
    if message.error().code() == KafkaError._PARTITION_EOF:
      continue
    else:
      print(f"Error: {message.error()}")
      break
  process_message(message)

# Publish network traffic data to Kafka topic def publish_to_kafka(source_ip, destination_ip, bytes_sent):
producer.produce(kafka_topic, f"{source_ip},{destination_ip},{bytes_sent}".encode('utf-8'))
producer.flush()

# Run the Streamlit app
if __name__ == '__main__':
  read_from_kafka()
